## Libraries

In [1]:
!pip install --upgrade dcor

In [2]:
!pip install minepy

In [3]:
import numpy as np
import pandas as pd
import random

#saves variables
import pickle
#saves into files
from numpy import savetxt

# # MIC correlation
# from minepy import MINE

# # distance correlation
# import dcor
from scipy.spatial.distance import correlation


import matplotlib.pyplot  as plt
import seaborn as sns
# sns.set()

# LSTM
from keras import metrics
import keras
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve, auc

from scipy.stats import multivariate_normal

In [4]:
class ADClass:

    def __init__(self, dataset_path):
        """
        Initialize dataset file path.
        """
        self.dataset_path = dataset_path
        self.LCVs = []
        #self.df_categorical_fields = ['MV101', 'P101', 'P102', 'MV201', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'MV301',
         #                             'MV302', 'MV303', 'MV304', 'P301', 'P302', 'P401', 'P402', 'P403', 'P404', 'UV401', 'P501',
        #                            'P502', 'P601', 'P602', 'P603']
        self.df_numerical_fields = ['P1_B2004', 'P1_B2016', 'P1_B3004', 'P1_B3005', 'P1_B4002', 'P1_B4005', 'P1_B400B', 'P1_B4022',
                                    'P1_FCV01D', 'P1_FCV01Z', 'P1_FCV02D', 'P1_FCV02Z', 'P1_FCV03D', 'P1_FCV03Z', 'P1_FT01', 'P1_FT01Z', 'P1_FT02',
                                    'P1_FT02Z', 'P1_FT03', 'P1_FT03Z', 'P1_LCV01D', 'P1_LCV01Z', 'P1_LIT01', 'P1_PCV01D', 'P1_PCV01Z',
                                    'P1_PCV02Z', 'P1_PIT01', 'P1_PIT02', 'P1_TIT01', 'P1_TIT02', 'P2_24Vdc', 'P2_SD01',
                                    'P2_SIT01', 'P2_VT01e', 'P2_VXT02', 'P2_VXT03', 'P2_VYT02', 'P2_VYT03', 'P3_LCP01D', 'P3_LCV01D',
                                     'P3_LT01', 'P4_HT_FD', 'P4_HT_LD', 'P4_HT_PO','P4_LD', 'P4_ST_FD',
                                    'P4_ST_LD', 'P4_ST_PO', 'P4_ST_PS', 'P4_ST_PT01', 'P4_ST_TT01']





    def importDataset(self, file_name, nb_rows):
        if nb_rows == 0:
          return pd.read_excel(self.dataset_path + file_name, header=1)
        else:
          return pd.read_excel(self.dataset_path + file_name, header=1, nrows=nb_rows)

    def trimColumnName(slfe, df):

        return df.rename(columns=lambda x: x.strip())

    def splitDataset(self, df):
        x_n, y_n = df.iloc[:, :-1], df.iloc[:, [-1]]

        return  x_n, y_n

    def oneHotEncoding(self, fixed_value_list, x_n):
        new_cat_list = list(filter(lambda x: x not in fixed_value_list, self.df_categorical_fields))

        #creating instance of one-hot-encoder
        encoder = OneHotEncoder(handle_unknown='ignore')
        # One-hot-encode the categorical columns.
        enc_package_type = pd.DataFrame(encoder.fit_transform(x_n[new_cat_list]).toarray())
        #merge one-hot encoded columns back with original DataFrame
        x_n = x_n.join(enc_package_type)

        return x_n

    def removeCategorical(self, fixed_value_list, x_n):
        new_cat_list = list(filter(lambda x: x not in fixed_value_list, self.df_categorical_fields))
        x_n.drop(new_cat_list, axis=1, inplace=True)

        return x_n


    def createSequence(slef, x_n_scaled, window_size, shift):
        # Create sequences
        train_windows = []
        for i in range(0, x_n_scaled.shape[0] - window_size - shift, shift):
            train_windows.append(x_n_scaled[i:i+window_size])

        # Convert to numpy array
        x_w_train = np.array(train_windows)

        return x_w_train

    def windowDetection_LSTM_AE(self, window_size_list, lstm_X_train, lstm_X_test, LSTM_w_shift):
        history_list = []
        mse_test = []

        for i in window_size_list:
          window_size = i

          # Create sequences
          train_windows = []
          test_windows = []

          x_w_train = AD_normal.createSequence(lstm_X_train, window_size, LSTM_w_shift)
          x_w_test = AD_normal.createSequence(lstm_X_test, window_size, LSTM_w_shift)

          # data dimensions // hyperparameters
          input_dim = x_w_train.shape[2]
          BATCH_SIZE = 256
          EPOCHS = 50

          # create model
          LSTM_model = keras.Sequential()
          LSTM_model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', batch_input_shape=(None, window_size, input_dim), return_sequences=True, name='encoder_1'))
          LSTM_model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='encoder_2'))
          LSTM_model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=False, name='encoder_3'))
          LSTM_model.add(keras.layers.RepeatVector(window_size, name='encoder_decoder_bridge'))
          LSTM_model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=True, name='decoder_1'))
          LSTM_model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='decoder_2'))
          LSTM_model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', return_sequences=True, name='decoder_3'))
          LSTM_model.add(keras.layers.TimeDistributed(keras.layers.Dense(input_dim)))

          early_stop = tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              min_delta=1e-2,
              patience=5,
              verbose=1, mode='auto',
              baseline=None,
              restore_best_weights=True
          )


          # the default learning rate is used for the Adam optimizer, which is typically set to 0.001.
          LSTM_model.compile(optimizer="adam",
                              loss="mse",
                              metrics=["acc"])

          # print an overview of our model
          LSTM_model.summary();

          history = LSTM_model.fit(
            x_w_train,
            x_w_train,
            epochs=50,
            batch_size=128,
            validation_split=0.2,
            callbacks=[
                early_stop
            ],
          )

          plt.plot(history.history["loss"], label="Training Loss")
          plt.plot(history.history["val_loss"], label="Validation Loss")
          plt.legend()
          plt.show()

          # Generate predictions on the test set
          X_w_pred = LSTM_model.predict(x_w_test)

          # Evaluate the autoencoder
          mse = np.mean(np.power(x_w_test - X_w_pred, 2), axis=1)
          print("Mean Squared Error:", np.mean(mse))
          print()

          # save values
          history_list.append(history)
          mse_test.append(np.mean(mse))

        return history_list, mse_test


    # /*
    # transposing the window array is necessary to ensure that each row represents a
    # feature and each column represents a sample, which is the correct format for
    # calculating the correlation matrix using the np.corrcoef function.

    # NOTE:
    # -----
    # Solutions to avoin NaN from deviding by zero
    # 1- To avoid the divide-by-zero problem when dealing with constant values.
    #    One such coefficient is the Spearman rank correlation coefficient.
    # 2- Add small amount of noise to the data.
    # */
    def calculateLCV(self, x_w_train):

        LCVs = []



        # sol.2 :
        for i in range(x_w_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[i])

          # find constant columns using NumPy isclose() function
          constant_cols = [col for col in window_df.columns if np.isclose(window_df[col], window_df[col].iloc[0], rtol=1e-15, atol=1e-15).all()]

          # Print the constant columns in the window
          # print(f"Constant columns in 2nd window {i//window_size}: {constant_cols}")

          # Add a small amount of random noise to the constant columns
          window_df[constant_cols] += np.abs(np.random.normal(scale=0.00000001, size=window_df[constant_cols].shape))

          # Compute the correlation matrix for the window
          corr_matrix = window_df.corr()

          # Extract the upper triangle of the correlation matrix into a vector
          correlation_vector = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]

          ### DEBUG
          # print(window_df)
          # print(corr_matrix)
          # print(correlation_vector)

          # from numpy import savetxt
          # # save to csv file
          # savetxt('window_df.csv', window_df, delimiter=',')
          # savetxt('corr_matrix_.csv', corr_matrix, delimiter=',')
          # savetxt('correlation_vector_.csv', correlation_vector, delimiter=',')

          LCVs.append(correlation_vector)
          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs

    def calculateLCV_spearman(self, x_w_train):

        LCVs = []


        for i in range(x_w_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[i])

          # find constant columns using NumPy isclose() function
          constant_cols = [col for col in window_df.columns if np.isclose(window_df[col], window_df[col].iloc[0], rtol=1e-15, atol=1e-15).all()]

          # Print the constant columns in the window
          # print(f"Constant columns in 2nd window {i//window_size}: {constant_cols}")

          # Add a small amount of random noise to the constant columns
          window_df[constant_cols] += np.abs(np.random.normal(scale=0.00000001, size=window_df[constant_cols].shape))

          # Compute the correlation matrix for the window
          corr_matrix = window_df.corr(method='spearman')

          # Extract the upper triangle of the correlation matrix into a vector
          correlation_vector = corr_matrix.values[np.triu_indices_from(corr_matrix, k=1)]

          LCVs.append(correlation_vector)
          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs

    def checkPosDef(self, x):

        return np.all(np.linalg.eigvals(x) > 0)

    def MGD(self, mean_vector, covariance_matrix):

        mgd = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

        return mgd

    def saveFile(self, file_name, var):
        # save to csv file
        savetxt(file_name, var, delimiter=',')

        return

    def calculateLCV_MIC(self, x_w_train):

        LCVs = []

        for k in range(x_w_train.shape[0]):

          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_w_train[k])
          cols = window_df.columns
          n_cols = len(cols)

          my_list = []

          for i in range(n_cols):
              for j in range(i+1, n_cols):
                  col1, col2 = cols[i], cols[j]
                  mine = MINE()
                  mine.compute_score(window_df[col1], window_df[col2])
                  my_list.append(mine.mic())

          my_array = np.array(my_list)

          LCVs.append(my_array)

          if not len(LCVs)%100:
            print(len(LCVs),'\n')

        return LCVs

    def calculateLCV_DistnaceCorr(self, x_w_train):


        LCVs = []
        n_cols = x_w_train.shape[2]

        for k in range(x_w_train.shape[0]):
            # Convert the window to a Pandas DataFrame
            window_df = pd.DataFrame(x_w_train[k])
            my_list = []

            for i in range(n_cols):
                for j in range(i+1, n_cols):
                    col1, col2 = window_df.columns[i], window_df.columns[j]
                    dcor_ = dcor.distance_correlation(window_df[col1], window_df[col2])
                    # dcor_ = correlation(window_df[col1], window_df[col2], w=None, centered=True)
                    my_list.append(dcor_)

            my_array = np.array(my_list)
            LCVs.append(my_array)

            if len(LCVs) % 100 == 0:
                print(len(LCVs), '\n')

        return LCVs


    def getLabels(self, x_y_train):

        LCVs_y = []

        for i in range(x_y_train.shape[0]):
          # Convert the window to a Pandas DataFrame
          window_df = pd.DataFrame(x_y_train[i])

          # Define the value you want to check for
          values_to_check = ['Attack', 'A ttack']

          # Check if the value exists in the 'B' column
          if window_df[0].isin(values_to_check).any():
              # print(window_df[1].values)
              LCVs_y.append(1)
          else:
              # print(window_df[0].values)
              LCVs_y.append(0)


        return LCVs_y

    def select_threshold(self, probs, test_data):
        best_epsilon = 0
        best_f1 = 0
        f = 0
        stepsize = (max(probs) - min(probs)) / 2000;
        epsilons = np.arange(min(probs), max(probs), stepsize)
        for epsilon in np.nditer(epsilons):
            predictions = (probs < epsilon)
            predictions
            f = f1_score(test_data, predictions, average='binary')
            # r = recall_score(test_data, predictions, average='binary')
            # Create confusion matrix

            if f > best_f1:
                best_f1 = f
                best_epsilon = epsilon

        # confusion matrix
        predictions = (probs < best_epsilon)
        cm = confusion_matrix(test_data, predictions)

        return best_f1, best_epsilon, cm

    def predictWithEpsilon(self, X, mvn, best_epsilon):
        predictions = []

        pdf = mvn.logpdf(X)
        predictions = (pdf < best_epsilon)

        return predictions, pdf

    def predictWithAlpha(self, LCV_list, Sigma, mu, alpha):
        predictions = []

        lower_bound = mu - alpha * np.diag(Sigma)
        upper_bound = mu + alpha * np.diag(Sigma)

        for LCV in LCV_list:
          if (mu - alpha * np.diag(Sigma) <= LCV).all() and (LCV <= mu + alpha * np.diag(Sigma)).all():
              predictions.append(0)
          elif (LCV < lower_bound).any() or (LCV > upper_bound).any():
              predictions.append(1)

        return predictions

    def findBestAlpha(self, LCVs, ground_truth, Sigma, mu):

        best_f1 = 0
        best_alpha = 0
        best_cm = None

        alpha = 0
        step = 1
        # for alpha in range(0, 50, 0.2):
        while alpha < 250:
            predictions = self.predictWithAlpha(LCVs, Sigma, mu, alpha)
            f1 = f1_score(ground_truth, predictions, average='binary')

            if f1 > best_f1:
                best_f1 = f1
                best_alpha = alpha

            alpha += step

        # confusion matrix
        predictions = self.predictWithAlpha(LCVs, Sigma, mu, best_alpha)
        cm = confusion_matrix(ground_truth, predictions)

        return best_f1, best_alpha, cm


### Import dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [6]:
%cd '/content/drive/My Drive/Colab Notebooks/'
#%pwd
# # from AD_Class import ADClass

/content/drive/My Drive/Colab Notebooks


In [7]:
dataset_path = 'My Drive/Colab Notebooks/'

In [35]:
df1=pd.read_csv('train1.csv', sep=';' )
df2=pd.read_csv('train2.csv', sep=';' )
df3=pd.read_csv('test1.csv', sep=';' )
df4=pd.read_csv('test2.csv', sep=';' )

frames_train = [df1,df2]
frames_test=[df3,df4]
df_concat_train=pd.concat(frames_train)
df_concat_test=pd.concat(frames_test)
#print("The dataset has {} records".format(len(df_concat)))
df_concat_train.head(5)
df_concat_test.head(5)

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_FD,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
0,2019-10-29 11:00:00,0.0982,1.4610,461.9883,1099.7363,32.0,0.0,113.4112,35.9070,9.2526,...,-0.0027,355.4688,349.6998,0.0,10053.0,27629.0,0,0,0,0
1,2019-10-29 11:00:01,0.0982,1.4578,461.9883,1099.7363,32.0,0.0,116.3903,35.8990,9.2161,...,-0.0020,356.8432,349.8625,0.0,10053.0,27629.0,0,0,0,0
2,2019-10-29 11:00:02,0.0982,1.4666,461.9883,1099.7363,32.0,0.0,116.3903,35.9211,9.8404,...,-0.0019,357.1868,350.4413,0.0,10053.0,27629.0,0,0,0,0
3,2019-10-29 11:00:03,0.0982,1.4688,461.9883,1099.7363,32.0,0.0,74.6806,35.9264,9.9271,...,-0.0035,357.9825,350.6583,0.0,10053.0,27629.0,0,0,0,0
4,2019-10-29 11:00:04,0.0982,1.4717,461.9883,1099.7363,32.0,0.0,67.7290,35.9337,9.8901,...,0.0018,358.2176,352.4487,0.0,10053.0,27629.0,0,0,0,0


In [36]:
# df_concat_train.to_excel (r'/content/drive/MyDrive/Colab Notebooks/df_concat_train.xlsx', index = None, header=True)


In [37]:
# # Get the list of all column names from headers
# column_headers = list(df4.columns.values)
# print("The Column Header :", column_headers)

In [38]:
# drop the columns for training that are under attack in each stage.
df1=df_concat_train.drop(['attack_P1','attack_P2','attack_P3'],axis=1)#, inplace=True)

In [39]:
# drop the columns for testing that are under attack in each stage.
df2=df_concat_test.drop(['attack_P1','attack_P2','attack_P3'],axis=1)#, inplace=True)

In [40]:
# df1.to_excel (r'/content/drive/MyDrive/Colab Notebooks/df_concat_train.xlsx', index = None, header=True)

In [41]:
# df2.to_excel (r'/content/drive/MyDrive/Colab Notebooks/df_concat_train.xlsx', index = None, header=True)

In [42]:
print(df1.shape)
print(df2.shape)

(550800, 61)
(444600, 61)


In [43]:
# head sample data for training only
df1.head()

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_HT_PO,P4_HT_PS,P4_LD,P4_ST_FD,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack
0,2019-09-11 20:00:00,0.0983,1.0702,399.2321,1110.3986,32.0,0.0,28.9981,35.7395,0.0,...,57.8704,0,395.0195,-0.0004,330.6568,328.9388,50.9871,9973.0,27629.0,0
1,2019-09-11 20:00:01,0.0983,1.0699,399.2321,1110.3986,32.0,0.0,29.9914,35.7388,0.0,...,58.4853,0,398.3109,-0.0012,330.2228,329.3728,50.9871,9973.0,27629.0,0
2,2019-09-11 20:00:02,0.0983,1.0703,399.2321,1110.3986,32.0,0.0,27.0122,35.7399,0.0,...,59.2629,0,397.8768,0.0018,332.0493,329.7526,50.9871,9973.0,27629.0,0
3,2019-09-11 20:00:03,0.0983,1.0719,399.2321,1110.3986,32.0,0.0,32.9705,35.7452,0.0,...,59.8235,0,398.6364,0.0033,331.5249,331.0547,50.9871,9973.0,27629.0,0
4,2019-09-11 20:00:04,0.0983,1.0710,399.2321,1110.3986,32.0,0.0,30.9843,35.7422,0.0,...,60.3118,0,401.2406,0.0015,333.1706,331.1631,50.9871,9973.0,27629.0,0


In [44]:
#Count 1 unique values in Training sample
#Count 1 unique values in each columns
df1_train=df1.nunique().reset_index()
df1_train.columns = ['feature','number Of unique']
df1_train[df1_train['number Of unique']==1]

,feature,number Of unique
26,P1_PCV02D,1
33,P2_Auto,1
34,P2_Emgy,1
35,P2_On,1
38,P2_TripEx,1
46,P3_LH,1
47,P3_LL,1
52,P4_HT_PS,1


In [45]:
# Trainig data before removing the unique features
print(df1.shape)

(550800, 61)


In [46]:
# drop the unique values in the dataset in trainig dataset.
df1_train=df1.drop(['P1_PCV02D','P2_Auto','P2_Emgy','P2_On','P2_TripEx','P3_LH','P3_LL','P4_HT_PS'], axis=1)

In [47]:
# Training data after removing the unique features
print(df1_train.shape)

(550800, 53)


In [48]:
#Count unique values in Test sample
df2_test=df2.nunique().reset_index()
df2_test.columns = ['feature','number Of unique']
df2_test[df2_test['number Of unique']==1]

,feature,number Of unique
33,P2_Auto,1
34,P2_Emgy,1
35,P2_On,1
36,P2_SD01,1
38,P2_TripEx,1
46,P3_LH,1
47,P3_LL,1
52,P4_HT_PS,1


In [49]:
# drop the unique values in the dataset in trainig dataset.
df2_test=df2.drop(['P2_Auto','P2_Emgy','P2_On','P2_SD01','P2_TripEx','P3_LH','P3_LL','P4_HT_PS'], axis=1)

In [52]:
# test data after removing the unique features
print(df2_test.shape)
print(df1_train.shape)

(444600, 53)
(550800, 53)


In [53]:
# count_row = df_concat_train.shape[0]  # Gives number of rows
# count_col = df.shape[1]  # Gives number of columns


df1_train=df1_train.iloc[350800:550800]
df2_test=df2_test.iloc[0:20000]
#df1_train.head(3)
#df2_test.head(3)
print(df2_test.shape)
print(df1_train.shape)


(20000, 53)
(200000, 53)


In [58]:
#df2_test['attack'].nunique()



In [21]:
# df1_train=df1_train.sample(frac=0.5, replace=True, random_state=1)

In [22]:
# df2_test=df2_test.sample(frac=0.5, replace=True, random_state=1)

In [62]:
#df1_train.to_excel(r'\content\drive\MyDrive\Colab Notebooks\train_all2.xlsx', index=False)
df1_train.to_excel (r'/content/drive/MyDrive/Colab Notebooks/train_all2.xlsx', index = None, header=True)


In [63]:
#df2_test.to_excel(r'\content\drive\MyDrive\Colab Notebooks\test_all2.xlsx', index=False)
df2_test.to_excel (r'/content/drive/MyDrive/Colab Notebooks/test_all2.xlsx', index = None, header=True)

/content/drive/MyDrive/Colab Notebooks


In [ ]:
AD_normal = ADClass(dataset_path)
AD_attack = ADClass(dataset_path)

In [ ]:
AD_normal